In [1]:
import sys
import warnings
warnings.filterwarnings("ignore")

import cv2
from keras.models import load_model
import numpy as np

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

In [3]:
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'

In [4]:
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')

In [5]:
image_path = '../../Dataset/img_align_celeba/000001.jpg'

In [6]:
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [8]:
from keras.preprocessing import image


In [17]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
def converter(x):

    #x has shape (batch, width, height, channels)
    return (0.21 * x[:,:,:1]) + (0.72 * x[:,:,1:2]) + (0.07 * x[:,:,-1:])

In [10]:
target_size=None
from skimage import io
from skimage.color import rgb2gray
from skimage import img_as_ubyte
rgb_img = io.imread(image_path)

In [81]:
import random
def gender_predict(img):
    gray_image=converter(img)
    rgb_image=np.squeeze(img)
    rgb_image = rgb_image.astype('uint8')
    
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,1)
        ret_arr=np.zeros((1,2))
        ret_arr[0][rand_index]=1.0
        return ret_arr
    x1, x2, y1, y2 = apply_offsets(faces[0], gender_offsets)
    
    rgb_face = rgb_image[y1:y2, x1:x2]
    rgb_face = cv2.resize(rgb_face, (gender_target_size))

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    
    return gender_classifier.predict(rgb_face)

In [20]:
import lime 

In [21]:
from lime import lime_image

In [22]:
explainer = lime_image.LimeImageExplainer()

In [28]:
import numpy as np
output_array=gender_predict(rgb_img)*100


(48, 48, 3)


{0: 'woman', 1: 'man'}

In [43]:
np.zeros((1,7)).astype('float32')

array([[0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [44]:
# display(image.load_img(image_path, target_size=None,color_mode='rgba
# image.img_to_array(test_img)
# x = np.expand_dims(rgb_img, axis=0)
# x.shape

In [29]:
explanation = explainer.explain_instance(rgb_img, gender_predict, top_labels=1, hide_color=0, num_samples=1000,batch_size=1)

In [67]:
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
%matplotlib inline
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=1000, hide_rest=False)

In [88]:
np.set_printoptions(threshold=sys.maxsize)

In [89]:
print(mask)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 

In [73]:
import os

In [85]:
# file='002715.jpg'
# rgb_img=io.imread(os.path.join(data_path,file))
# gender_predict(rgb_img)

In [76]:
data_path='../../Dataset/img_align_celeba/'
output_dict={}
for file in os.listdir(data_path):
    rgb_img = io.imread(os.path.join(data_path,file))
    try:
        output=gender_predict(rgb_img)[0]
    except:
        print(file)
    output_dict[file]={gender_labels[i]:np.round(output[i]*100,2) for i in range(len(gender_labels))}

002715.jpg
005424.jpg
014893.jpg
016986.jpg
023984.jpg
031787.jpg
036260.jpg
041431.jpg
044120.jpg
052332.jpg
053685.jpg
057093.jpg
070428.jpg
076224.jpg
078874.jpg
080583.jpg
081789.jpg
096776.jpg
101170.jpg
103761.jpg
104520.jpg
105255.jpg
106158.jpg
119593.jpg
121173.jpg
121224.jpg
123602.jpg
125060.jpg
126589.jpg
127854.jpg
133832.jpg
139631.jpg
142416.jpg
142996.jpg
144243.jpg
151721.jpg
157974.jpg
159541.jpg
161275.jpg
167056.jpg
169444.jpg
178911.jpg
178983.jpg
186886.jpg
196665.jpg


In [77]:
import json

with open('gender_results.json', 'w') as fp:
    json.dump(output_dict, fp)

In [83]:
genderkeys=list(output_dict.keys())
for i in range(len(genderkeys)):
    if i==1000:
        break
    print(genderkeys[i],output_dict[genderkeys[i]])

000001.jpg {'woman': 94.58, 'man': 5.42}
000002.jpg {'woman': 91.77, 'man': 8.23}
000003.jpg {'woman': 0.0, 'man': 100.0}
000004.jpg {'woman': 100.0, 'man': 0.0}
000005.jpg {'woman': 39.16, 'man': 60.84}
000006.jpg {'woman': 93.49, 'man': 6.51}
000007.jpg {'woman': 15.08, 'man': 84.92}
000008.jpg {'woman': 32.18, 'man': 67.82}
000009.jpg {'woman': 95.43, 'man': 4.57}
000010.jpg {'woman': 98.05, 'man': 1.95}
000011.jpg {'woman': 94.65, 'man': 5.35}
000012.jpg {'woman': 31.56, 'man': 68.44}
000013.jpg {'woman': 16.24, 'man': 83.76}
000014.jpg {'woman': 93.04, 'man': 6.96}
000015.jpg {'woman': 7.0, 'man': 93.0}
000016.jpg {'woman': 58.27, 'man': 41.73}
000017.jpg {'woman': 94.62, 'man': 5.38}
000018.jpg {'woman': 98.31, 'man': 1.69}
000019.jpg {'woman': 97.55, 'man': 2.45}
000020.jpg {'woman': 89.84, 'man': 10.16}
000021.jpg {'woman': 6.0, 'man': 94.0}
000022.jpg {'woman': 98.34, 'man': 1.66}
000023.jpg {'woman': 5.98, 'man': 94.02}
000024.jpg {'woman': 95.82, 'man': 4.18}
000025.jpg {'wo

000601.jpg {'woman': 88.73, 'man': 11.27}
000602.jpg {'woman': 16.51, 'man': 83.49}
000603.jpg {'woman': 100.0, 'man': 0.0}
000604.jpg {'woman': 52.04, 'man': 47.96}
000605.jpg {'woman': 98.88, 'man': 1.12}
000606.jpg {'woman': 96.31, 'man': 3.69}
000607.jpg {'woman': 90.47, 'man': 9.53}
000608.jpg {'woman': 77.65, 'man': 22.35}
000609.jpg {'woman': 93.7, 'man': 6.3}
000610.jpg {'woman': 58.29, 'man': 41.71}
000611.jpg {'woman': 98.46, 'man': 1.54}
000612.jpg {'woman': 97.2, 'man': 2.8}
000613.jpg {'woman': 98.06, 'man': 1.94}
000614.jpg {'woman': 0.0, 'man': 100.0}
000615.jpg {'woman': 96.09, 'man': 3.91}
000616.jpg {'woman': 71.67, 'man': 28.33}
000617.jpg {'woman': 9.78, 'man': 90.22}
000618.jpg {'woman': 81.89, 'man': 18.11}
000619.jpg {'woman': 7.53, 'man': 92.47}
000620.jpg {'woman': 0.0, 'man': 100.0}
000621.jpg {'woman': 96.06, 'man': 3.94}
000622.jpg {'woman': 64.29, 'man': 35.71}
000623.jpg {'woman': 0.0, 'man': 100.0}
000624.jpg {'woman': 4.36, 'man': 95.64}
000625.jpg {'wom

In [84]:
except_list=['002715.jpg',
'005424.jpg',
'014893.jpg',
'016986.jpg',
'023984.jpg',
'031787.jpg',
'036260.jpg',
'041431.jpg',
'044120.jpg',
'052332.jpg',
'053685.jpg',
'057093.jpg',
'070428.jpg',
'076224.jpg',
'078874.jpg',
'080583.jpg',
'081789.jpg',
'096776.jpg',
'101170.jpg',
'103761.jpg',
'104520.jpg',
'105255.jpg',
'106158.jpg',
'119593.jpg',
'121173.jpg',
'121224.jpg',
'123602.jpg',
'125060.jpg',
'126589.jpg',
'127854.jpg',
'133832.jpg',
'139631.jpg',
'142416.jpg',
'142996.jpg',
'144243.jpg',
'151721.jpg',
'157974.jpg',
'159541.jpg',
'161275.jpg',
'167056.jpg',
'169444.jpg',
'178911.jpg',
'178983.jpg',
'186886.jpg',
'196665.jpg']